In [1]:
import os
from dotenv import load_dotenv
from getpass import getpass

# Load environment variables from .env file
load_dotenv()

# Get API base URL and model with default values
OPENAI_MODEL = os.getenv("OPENAI_MODEL", "gpt-4o-mini")
OLLAMA_API_BASE = os.getenv("OLLAMA_API_BASE", "http://localhost:11434/v1")
OLLAMA_API_KEY = os.getenv("OLLAMA_API_KEY", "ollama")
OLLAMA_MODEL = os.getenv("OLLAMA_MODEL", "qwen2.5:7b")

# Set up OpenAI API configuration
# Try to get API key from environment variables
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
# If not found, ask user to input it
if not OPENAI_API_KEY:
    OPENAI_API_KEY = getpass("Enter your OpenAI API key:")

from agents import OpenAIChatCompletionsModel, AsyncOpenAI

gpt4om_model= OpenAIChatCompletionsModel(
    model="gpt-4o-mini",
    openai_client=AsyncOpenAI(api_key=OPENAI_API_KEY),
)

qwen_model= OpenAIChatCompletionsModel(
    model="qwen2.5:7b",
    openai_client=AsyncOpenAI(base_url=OLLAMA_API_BASE,api_key=OLLAMA_API_KEY),
)

import json
from agents import function_tool

def get_form_schema() -> list:
    """Get the schema definition of the form data"""
    return [
        {
            "name": "username",
            "type": "string",
            "required": True,
            "description": "User name, between 3 and 20 characters long",
            "value": None,
        },
        {
            "name": "age",
            "type": "number",
            "required": True,
            "description": "Age, please enter a positive integer",
            "value": None,
        },
        {
            "name": "email",
            "type": "string",
            "required": False,
            "description": "Email address to receive notifications",
            "value": None,
        }
    ]

def ensure_form_file_exists(file_path: str) -> bool:
    """
    Ensure the form data file exists, create it if it doesn't
    """
    try:
        if not os.path.exists(file_path):
            print(f"Form data file does not exist: {file_path}, creating new file")
            # Ensure directory exists
            os.makedirs(os.path.dirname(file_path), exist_ok=True)
            # Create empty form data
            empty_form_data = get_form_schema()
            # Write empty data to file
            with open(file_path, 'w', encoding='utf-8') as f:
                json.dump(empty_form_data, f, ensure_ascii=False, indent=2)
        return True
    except Exception as e:
        print(f"Error creating form data file: {e}")
        return False

@function_tool
def get_form_data_tool() -> str:
    """
    Read the form data filled by the user
    """
    file_path = "./form_data.json"
    try:
        if not ensure_form_file_exists(file_path):
            return get_form_schema()

        with open(file_path, 'r', encoding='utf-8') as f:
            form_data = json.load(f)
            return json.dumps(form_data, indent=2)
            # return form_data
    except json.JSONDecodeError:
        return(f"JSON format error: {file_path}")
    except Exception as e:
        return(f"Error reading form data: {e}")

@function_tool
def update_form_field_tool(field_name:str, field_value: str|int|float) -> str:
    """
    Update a field in the form data

    Args:
        field_name (str): Form field name
        field_value (str): Form field value
    
    Returns:
        Updated form data
    """
    file_path = "./form_data.json"
    try:
        # Ensure file exists
        if not ensure_form_file_exists(file_path):
            return
            
        with open(file_path, 'r+', encoding='utf-8') as f:
            form_data = json.load(f)
            # Find the field to update
            for field in form_data:
                if field['name'] == field_name:
                    # Convert value according to field type
                    if field['type'] == 'number':
                        try:
                            field['value'] = int(field_value)
                        except ValueError:
                            try:
                                field['value'] = float(field_value)
                            except ValueError:
                                return(f"Warning: Cannot convert '{field_value}' to number type, keeping original string")
                                # field['value'] = field_value
                    else:  # string or other types
                        field['value'] = field_value
                    break
            f.seek(0)
            json.dump(form_data, f, ensure_ascii=False, indent=2)
            f.truncate()
        return f"Form data update successful: {field_name} = {field_value}"
    except Exception as e:
        return(f"Error updating form data: {e}")
    
@function_tool
def get_quote_tool(username: str, age: int, email: str, term: int, amount: int ) -> str:
    """
    Get a insurers quote based on user information

    Args:
        username (str): User name
        age (int): User age
        term (int): Protection term in years
        amount (int): Loan amount in dollars
    """

    # Age     Monthly     Annual
    # 35 - 54 $ 28.80     $ 320.00
    # 55 - 74 $ 333.90    $ 3,710.00
    # 75 - 89 $ 727.20    $ 8,080.00

    monthly_payment = 0
    annual_payment = 0 
    if age >= 35 and age <= 54:
        monthly_payment = 28.80
        annual_payment = 320.00
    elif age >= 55 and age <= 74:
        monthly_payment = 333.90
        annual_payment = 3710.00
    elif age >= 75 and age <= 89:
        monthly_payment = 727.20
        annual_payment = 8080.00

    quote = {
        "username": username,
        "age": age,
        "amount": amount,
        "Monthly Payment": monthly_payment,
        "Annual Payment": annual_payment,
    }
    return f"Hello {username}, you are {age} years old. A quote for you: 'The only limit to our realization of tomorrow is our doubts of today.'"

In [2]:
from agents import Agent

form_agent = Agent(
    name="User information Assistant",
    instructions="You are an assistant who helps users fill out forms to save user information. Please query and update the user's information based on the user's input.",
    tools=[
        get_form_data_tool,
        update_form_field_tool
    ],
    model=qwen_model,
    # model=gpt4om_model,
)

quote_agent = Agent(
    name="Quote Assistant",
    instructions="You are an assistant helping users get a insurers quote based on their information.",
    tools=[
        get_quote_tool
    ],
    model=qwen_model,
    # model=gpt4om_model,
)

triage_agent = Agent(
    name="Triage Assistant",
    instructions=(
        "Help the user with their questions."
        "If they ask for their information, handoff to the form agent."
        "If they ask for a quote, handoff to the quote function."
    ),
    handoffs=[form_agent, quote_agent],
    model=qwen_model,
)

In [3]:
from agents import Runner, RunResultStreaming
from openai.types.responses import (
    ResponseTextDeltaEvent,
    ResponseFunctionCallArgumentsDeltaEvent,
)

async def show_streaming_response(
    response: RunResultStreaming,
    verbose: bool = False,
) -> None:
    async for event in response.stream_events():
        # print(f'\n{event.type}')
        # print(event)
        if event.type == "raw_response_event": 
            if isinstance(event.data, ResponseFunctionCallArgumentsDeltaEvent) and verbose:
                # this is streamed parameters for our tool call
                print(event.data.delta, end="", flush=True)
            elif isinstance(event.data, ResponseTextDeltaEvent):
                # this is streamed final answer tokens
                print(event.data.delta, end="", flush=True)
        elif event.type == "agent_updated_stream_event" and verbose:
            # this tells us which agent is currently in use
            print(f"> Current Agent: {event.new_agent.name}")
        elif event.type == "run_item_stream_event" and verbose:
            # these are events containing info that we'd typically
            # stream out to a user or some downstream process
            if event.name == "tool_called":
                # this is the collection of our _full_ tool call after our tool
                # tokens have all been streamed
                print()
                print(f"> Tool Called, name: {event.item.raw_item.name}")
                print(f"> Tool Called, args: {event.item.raw_item.arguments}")
            elif event.name == "tool_output":
                # this is the response from our tool execution
                print(f"> Tool Output: {event.item.raw_item['output']}")
        elif verbose:
            print(f"Unknown event type: {event.type}", type(event).__name__)
            print(event)

# Define input_list as a global variable to maintain conversation history
input_list = []

async def handle_customer_query(query: str) -> None:
    global input_list
    
    # Create the response using the current input list plus the new query
    response = Runner.run_streamed(
        starting_agent=triage_agent,
        input=input_list + [{"role": "user", "content": query}]
    )
    
    # Show the streaming response
    await show_streaming_response(response)
    
    # Update the input list for the next conversation turn
    input_list = response.to_input_list()

In [4]:
await handle_customer_query("List all my information")

Here is your current information:

- Username: Not set
- Age: Not set
- Email: Not set

Would you like to update any of these details?

In [6]:
await handle_customer_query("My Name is John Doe. What other information do I need to provide?")

Hello John Doe! To continue, we just need your age and, optionally, an email address for notifications.

Could you please provide:

- Your age (as a positive integer)
- Your email address (if you want us to send notifications)

In [7]:
await handle_customer_query("I'm 29.")

Great! And could you please provide your email address if you wish to receive any updates or notifications? If not, that's okay too.

In [9]:
await handle_customer_query("Yes. Let me see all my information.")

Sure, let's gather the information together:

- Username: Not explicitly set in this conversation (we can call you John Doe)
- Age: 29
- Email: (not provided yet)

Would you like to provide an email address now? If yes, please do so; otherwise, I'll proceed without it.

In [10]:
await handle_customer_query("Please make me a term 20 insurers quote.")

Sure, I can help with that. Could you please provide the amount for which you would like to get a quote? For example, how much coverage are you interested in purchasing? This will be the "amount" parameter needed to generate your quote.

If you're not sure about the exact amount, we can use an estimate or discuss what might be appropriate based on your needs.

In [11]:
await handle_customer_query("The amount is $800000.")

Perfect! We have your request to get a term 20 insurance quote for an amount of $800,000.

Could you also provide any additional details such as your preferred insurer or specific terms that might affect the policy (e.g., health status, age, etc.)? This information will help us to tailor the quotes more accurately.
If no further details are needed, we can proceed with the standard evaluation process.

In [12]:
await handle_customer_query("You can use abc@abc.com")

Got it! Using abc@abc.com as your email address for notifications.

Now, let's proceed with generating the quote for term 20 insurance coverage of $800,000.

Do you have any additional details or specific requirements that should be considered? If not, we can move forward with a standard evaluation.

In [14]:
await handle_customer_query("请保存我的个人信息.")

Sure! I'll save your personal information.

- Username: John Doe (extracted from your full name)
- Age: 29
- Email: abc@abc.com

Would you also like to set a username specifically for our services or proceed with "John Doe"? Additionally, do you have any other details you'd like to add?